# Base Model

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B", device_map="auto", load_in_4bit=True)

def generate_response(prompt, model):
    messages = [
        {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
        {"role": "user", "content": f"{prompt}"},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [2]:
instruction = "피타고라스 정리에 대해 알려줘"

generate_response(instruction, model)

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\bitsandbytes\nn\modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


'피타고라스 정리는 두 정사각형의 넓이가 같을 때, 그들의 대각선의 길이의 비는 정수라는 것을 보여준다.'

In [3]:
instruction = "건강을 유지하는 3가지 팁 알려줘"

generate_response(instruction, model)

'건강을 유지하는 3가지 팁 알려줘'

In [4]:
instruction = "금리가 오르면 물가가 어떻게 돼?"

generate_response(instruction, model)

'금리가 오르면 물가가 어떻게 돼?'

In [5]:
instruction = "사과 10개를 사서 냉장고에 넣고 1개를 먹었어 그럼 사과는 몇개가 남아있지?"

generate_response(instruction, model)

'사과 10개에서 1개를 빼면 9개가 남아있지.assistant\n\n사과 10개에서 1개를 빼면 9개가 남아있지.assistant'

In [6]:
instruction = "달러의 환율이 오르면 생기는 일 알려줘"

generate_response(instruction, model)

'달러의 환율이 오르면 생기는 일 알려줘.'

In [7]:
instruction = "K팝에 대해 알려줘"

generate_response(instruction, model)

'K-POP에 대해 알려줘.'

# Instruct-tuned Model

In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("gamzadole/llama3_instruct_tuning_without_pretraing")
base_model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B", device_map="auto", load_in_4bit=True)
model = PeftModel.from_pretrained(base_model, "gamzadole/llama3_instruct_tuning_without_pretraing")
tokenizer = AutoTokenizer.from_pretrained("gamzadole/llama3_instruct_tuning_without_pretraing")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
alpaca_prompt = """아래는 질문 instruction 과 추가정보를 나타내는 input 입니다. 적절한 response를 생성해주세요.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{response}"""

In [3]:
def generate_response(prompt, model):
    prompt = alpaca_prompt.format(instruction=prompt, input="", response="")
    messages = [
        {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
        {"role": "user", "content": f"{prompt}"},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [10]:
instruction = "피타고라스 정리에 대해 알려줘"

generate_response(instruction, model)

'피타고라스 정리는 직각 삼각형의 두 변의 길이와 그 직각 삼각형의 높이의 곱이 같다는 것입니다.'

In [11]:
instruction = "건강을 유지하는 3가지 팁 알려줘"

generate_response(instruction, model)

'건강을 유지하는 세 가지 팁은 다음과 같습니다.\n\n1. 적절한 수면 시간을 유지하세요.\n\n2. 적절한 식습관을 유지하세요.\n\n3. 규칙적인 운동을 하세요.'

In [12]:
instruction = "금리가 오르면 물가가 어떻게 돼?"

generate_response(instruction, model)

'금리가 오르면 물가는 상승합니다.'

In [8]:
instruction = "사과 10개를 사서 냉장고에 넣고 1개를 먹었어 그럼 사과는 몇개가 남아있지?"

generate_response(instruction, model)

'친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘.'

In [14]:
instruction = "달러의 환율이 오르면 생기는 일 알려줘"

generate_response(instruction, model)

'달러의 환율이 오르면 수입품의 가격이 상승하여 수입품의 수요가 감소하고, 수입품의 수입이 감소하여 국내 산업이 위축될 수 있습니다.'

In [16]:
instruction = "K팝에 대해 알려줘"

generate_response(instruction, model)

'K-POP은 대한민국에서 시작된 대중음악 장르입니다.'